# Generation-based MRC 문제를 풀어보기

+ 사전학습 모델을 불러오고, 이를 활용해 MRC 문제를 해결해봅니다.
+ 범용적으로 사용되는 다국어 모델인 mT5 모델을 이용해 Generation 기반 MRC를 위한 Fine-tuning을 진행합니다.
+ 학습된 모델로 추론하고, 평가지표를 통해 모델의 성능을 평가합니다.


### Requirements

In [ ]:
!pip install tqdm==4.64.1
!pip install datasets==2.15.0
!pip install transformers==4.36.1
!pip install sentencepiece==0.1.97
!pip install accelerate -U
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00


In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import numpy as np

In [37]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## 데이터 및 평가 지표 불러오기

In [18]:
from datasets import load_from_disk

datasets = load_from_disk('./resources/data/data_kosquadv1_train_dataset')

In [19]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 13556
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [20]:
from datasets import load_metric

metric = load_metric('squad')

## Pre-trained 모델 및 토크나이저 불러오기

In [21]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

In [22]:
model_name = "google/mt5-small"

In [23]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config,
    cache_dir=None,
)

## 설정하기

In [24]:
max_source_length = 384
max_target_length = 16
padding = "max_length"
preprocessing_num_workers = 12
num_beams = 3
max_train_samples = 5000
max_val_samples = 500
num_train_epochs = 5
train_batch_size = 16
eval_batch_size = 8
learning_rate = 1e-3

## 전처리하기

In [25]:
def preprocess_function(examples):
    inputs = [f'question: {q}  context: {c}' for q, c in zip(examples['question'], examples['context'])]
    targets = [f'{a["text"][0]}' for a in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors='pt')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100
    model_inputs["example_id"] = []
    for i in range(len(model_inputs["labels"])):
        model_inputs["example_id"].append(examples["id"][i])
    return model_inputs

In [26]:
column_names = datasets['train'].column_names

In [27]:
train_dataset = datasets["train"]
#train_dataset = train_dataset.select(range(max_train_samples))
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

Map (num_proc=12): 100%|██████████| 13556/13556 [00:03<00:00, 4177.93 examples/s] 


In [28]:
eval_dataset = datasets["validation"]
#eval_dataset = eval_dataset.select(range(max_val_samples))
eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )


Map (num_proc=12): 100%|██████████| 240/240 [00:00<00:00, 774.47 examples/s]


## Fine-tuning하기

In [29]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

In [30]:
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
        )

In [31]:
features = [train_dataset.remove_columns('example_id')[i] for i in range(5)]
examples = data_collator(features)
examples.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [32]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # decoded_labels is for rouge metric, not used for f1/em metric

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    formatted_predictions = [{"id": ex['id'], "prediction_text": decoded_preds[i]} for i, ex in enumerate(datasets["validation"].select(range(max_val_samples)))]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"].select(range(max_val_samples))]

    result = metric.compute(predictions=formatted_predictions, references=references)
    return result

In [33]:
args = Seq2SeqTrainingArguments(
    output_dir='outputs',
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    predict_with_generate=True,
    num_train_epochs=num_train_epochs,
    save_strategy = 'epoch',
    evaluation_strategy = 'epoch',
    save_total_limit = 2,
    logging_strategy = 'epoch',
    load_best_model_at_end = True,
    learning_rate = learning_rate,
    remove_unused_columns = True
)

In [34]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [38]:
train_result = trainer.train()

Epoch,Training Loss,Validation Loss


IndexError: Index 499 out of range for dataset of size 240.

In [36]:
!pip list

Package                  Version
------------------------ -----------
accelerate               1.0.1
aiohappyeyeballs         2.4.3
aiohttp                  3.10.10
aiosignal                1.3.1
asttokens                2.4.1
async-timeout            4.0.3
attrs                    24.2.0
bitsandbytes             0.44.1
certifi                  2024.8.30
charset-normalizer       3.4.0
click                    8.1.7
comm                     0.2.2
datasets                 2.15.0
debugpy                  1.8.7
decorator                5.1.1
dill                     0.3.7
docstring_parser         0.16
exceptiongroup           1.2.2
executing                2.1.0
faiss-gpu                1.7.2
filelock                 3.16.1
frozenlist               1.4.1
fsspec                   2023.10.0
huggingface-hub          0.26.0
idna                     3.10
ipykernel                6.29.5
ipython                  8.28.0
jedi                     0.19.1
Jinja2                   3.1.4
joblib         

## 평가하기

In [ ]:
metrics = trainer.evaluate(
    max_length=max_target_length, num_beams=num_beams, metric_key_prefix="eval"
)

In [ ]:
metrics

{'eval_loss': 0.6535626649856567,
 'eval_exact_match': 62.2,
 'eval_f1': 68.32095238095236,
 'eval_runtime': 28.9502,
 'eval_samples_per_second': 17.271,
 'eval_steps_per_second': 2.176,
 'epoch': 5.0}

In [ ]:
def generarate_answer(sample):
    inputs = f'question: {sample["question"]}  context: {sample["context"]} </s>'
    print(inputs)
    sample = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors='pt')
    sample = sample.to("cuda:0")
    outputs = model.generate(**sample, max_length=max_target_length, num_beams=num_beams)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

    pred = "\n".join(nltk.sent_tokenize(pred))

    return pred

np.random.seed(seed=7777)

for i in np.random.randint(0, len(datasets["validation"]), 5):
    print(generarate_answer(datasets["validation"][int(i)]))
    print("=" * 8)

question: 유아인이 배우로서 처음으로 부산국제영화제에 참석한 년도는?  context: 2006년 1월 스크린 데뷔작인 독립영화 《우리에게 내일은 없다》의 촬영을 시작했다. 이 영화를 연출한 노동석 감독은 오디션을 볼 당시 유아인에게 극 중 캐릭터에 대해 묻자 창 밖을 한참 바라보며 “슬프죠”라는 한 마디만을 던진 모습이 인상적이었다며 캐스팅의 이유를 밝혔다. 유아인은 이 영화에서 진짜 총을 구해 현실로부터 자신을 구해내려는 소년 ‘종대’ 역할을 맡았는데, 인터뷰에서 "종대처럼 사건에 휘말린 적도 없고 불우한 환경에서 자라지도 않았지만 제가 종대와 비슷한 시기에 느꼈던 불안이나 두려움 등이 연기를 하는 데 큰 도움이 됐습니다. 종대도 청춘이고 저도 청춘이니까요"라며 연기를 한 소회를 밝혔다. 2007년 5월 《우리에게 내일은 없다》 언론시사회에서는 작품에 대해 “배우라는 앞날에 대한 꿈을 꾸고 그림을 그렸다면 그 그림 속에 꼭 있어야 할 영화”라며 본인의 영화 데뷔작에 대한 애정을 드러낸다. 또한 배우로서 고유한 소년성을 갖게해 준 ‘첫 활시위’ 같은 작품이라고 설명한다. 2006년 10월 유아인은 이 영화를 통해 배우로서 처음으로 부산국제영화제 개막식과 GV에 참석한다. </s>
2006년
question: 김희선이 4년만에 브라운관에 컴백한 인기 드라마 <야마토 나데시코>를 원작으로 한 로맨스 드라마는?  context: 김희선은 2000년대에 접어들면서 스크린으로 활동 무대를 옮겨 드라마 출연을 한동안 중단하였다. 영화 《와니와 준하》(2001), 《화성으로 간 사나이》(2003)에 출연했지만 번번히 이렇다 할 흥행을 거두지 못한 채 2003년 일본의 인기 드라마 《야마토 나데시코》를 원작으로 한 로맨스 드라마 《요조숙녀》로 4년여만에 브라운관에 컴백하였다. 하지만 이 작품은 진부한 설정과 스토리로 기대 이상의 주목은 받지 못했다. 이듬해, 2004년에는 한류를 겨냥한 멜로 드라마 《슬픈 연가》에서 출연하였지만 남자주인공 중 한 명인 송승헌이 병역

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

